# 8 qubit

In [40]:
import pennylane as qml
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# Number of qubits
num_qubits = 8

# Set the overlap threshold
mu = 1  # Desired maximum overlap

# Define the quantum devices
dev1 = qml.device('default.qubit', wires=num_qubits)
dev2 = qml.device('default.qubit', wires=num_qubits)

# Define the quantum circuit for Group 1 (Entangled States)
@qml.qnode(dev1)
def generate_entangled_state(params):
    qml.templates.AngleEmbedding(params, wires=range(num_qubits))
    # Apply CNOT gates to introduce entanglement
    for i in range(num_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.state()

# Define the quantum circuit for Group 2 (Product States)
@qml.qnode(dev2)
def generate_product_state(params):
    qml.templates.AngleEmbedding(params, wires=range(num_qubits))
    # No entangling gates applied here
    return qml.state()

# Function to generate states with overlap control
def generate_states_with_overlap_control(myu, num_states=10):
    group1 = []  # Entangled states
    group2 = []  # Product states

    while len(group1) < num_states or len(group2) < num_states:
        # Generate random parameters within distinct ranges
        params1 = np.random.uniform(0, np.pi/4, num_qubits)  # Narrow range for entangled states
        params2 = np.random.uniform(3 * np.pi / 4, np.pi, num_qubits)  # Different range for product states

        # Generate states
        psi1 = generate_entangled_state(params1)
        psi2 = generate_product_state(params2)

        # Compute overlap
        overlap = np.abs(np.vdot(psi1, psi2))**2

        # Check if the overlap condition is satisfied
        if overlap < myu:
            if len(group1) < num_states:
                group1.append(psi1)
            if len(group2) < num_states:
                group2.append(psi2)

    return group1, group2


# Convert PennyLane state vector to Qiskit QuantumCircuit
def state_to_quantum_circuit(state, num_qubits):
    # Convert PennyLane tensor to a NumPy array
    state_np = np.array(state)

    # Create a new QuantumCircuit with the same number of qubits
    qc = QuantumCircuit(num_qubits)
    
    # Initialize the QuantumCircuit with the given state vector
    qc.initialize(state_np, range(num_qubits))
    
    return qc

# Generate the states with the desired overlap
entangled_states, product_states = generate_states_with_overlap_control(myu=mu, num_states=200)

# Convert each entangled and product state into a QuantumCircuit
entangled_qcs = [state_to_quantum_circuit(psi, num_qubits) for psi in entangled_states]
product_qcs = [state_to_quantum_circuit(psi, num_qubits) for psi in product_states]
'''
# Display the circuits
for i, qc in enumerate(entangled_qcs):
    print(f"Entangled Circuit {i + 1}:")
    print(qc)
    
for i, qc in enumerate(product_qcs):
    print(f"Product Circuit {i + 1}:")
    print(qc)
    '''


'\n# Display the circuits\nfor i, qc in enumerate(entangled_qcs):\n    print(f"Entangled Circuit {i + 1}:")\n    print(qc)\n    \nfor i, qc in enumerate(product_qcs):\n    print(f"Product Circuit {i + 1}:")\n    print(qc)\n    '

In [41]:
entangled_qcs

In [42]:
X_train = entangled_qcs + product_qcs

y_train = np.array([1] * len(entangled_qcs) + [0] * len(product_qcs))


In [43]:
X_train

In [44]:
# Generate the states with the desired overlap
entangled_states_test, product_states_test = generate_states_with_overlap_control(myu=mu, num_states=10)

# Convert each entangled and product state into a QuantumCircuit
entangled_qcs_test = [state_to_quantum_circuit(psi, num_qubits) for psi in entangled_states_test]
product_qcs_test = [state_to_quantum_circuit(psi, num_qubits) for psi in product_states_test]

X_test = entangled_qcs_test + product_qcs_test

y_test = np.array([1] * len(entangled_qcs_test) + [0] * len(product_qcs_test))


In [45]:
'''from qiskit import QuantumCircuit
from qiskit.quantum_info import random_statevector
import numpy as np

def create_entangled_state(num_qubits):
    qc = QuantumCircuit(num_qubits)
    qc.h(0)
    for i in range(1, num_qubits):
        qc.cx(0, i)
    return qc

def create_product_state(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        qc.h(i)
    return qc

def generate_dataset(num_qubits, num_samples):
    X, y = [], []
    for _ in range(num_samples):
        if np.random.rand() > 0.5:
            qc = create_entangled_state(num_qubits)
            y.append(0)
        else:
            qc = create_product_state(num_qubits)
            y.append(1)
        X.append(qc)
    return X, y

# Generate training and test sets
num_qubits = 4
num_samples = 10
X_train, y_train = generate_dataset(num_qubits, num_samples)
X_test, y_test = generate_dataset(num_qubits, num_samples)
'''

'from qiskit import QuantumCircuit\nfrom qiskit.quantum_info import random_statevector\nimport numpy as np\n\ndef create_entangled_state(num_qubits):\n    qc = QuantumCircuit(num_qubits)\n    qc.h(0)\n    for i in range(1, num_qubits):\n        qc.cx(0, i)\n    return qc\n\ndef create_product_state(num_qubits):\n    qc = QuantumCircuit(num_qubits)\n    for i in range(num_qubits):\n        qc.h(i)\n    return qc\n\ndef generate_dataset(num_qubits, num_samples):\n    X, y = [], []\n    for _ in range(num_samples):\n        if np.random.rand() > 0.5:\n            qc = create_entangled_state(num_qubits)\n            y.append(0)\n        else:\n            qc = create_product_state(num_qubits)\n            y.append(1)\n        X.append(qc)\n    return X, y\n\n# Generate training and test sets\nnum_qubits = 4\nnum_samples = 10\nX_train, y_train = generate_dataset(num_qubits, num_samples)\nX_test, y_test = generate_dataset(num_qubits, num_samples)\n'

In [46]:
from qiskit.circuit.library import EfficientSU2
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp, Pauli


def create_vqa_circuit(num_qubits):
    return EfficientSU2(num_qubits, reps=1)


# Initialize Estimator
estimator = AerEstimator()

def cost_function(params, estimator, circuits, labels):
    """
    Define the cost function for the VQA.

    Args:
        params (array): Parameters for the variational quantum circuit.
        estimator (Estimator): Quantum estimator to compute expectation values.
        circuits (list): List of quantum circuits representing the training data.
        labels (list): List of actual labels for the training data.

    Returns:
        float: The mean squared error between the expectation values and the labels.
    """
    cost = 0
    for qc, label in zip(circuits, labels):
        # Create the variational quantum circuit and assign parameters
        vqc = create_vqa_circuit(qc.num_qubits)
        vqc.assign_parameters(params, inplace=True)
        
        # Combine the training circuit with the variational circuit
        full_circuit = qc.compose(vqc)
        
        # Compute the expectation value of the Pauli-Z operator
        result = estimator.run([full_circuit], [SparsePauliOp("Z" * qc.num_qubits)]).result()
        expectation_value = result.values[0]
        
        # Calculate the squared difference between the expectation value and the label
        cost += (expectation_value - label) ** 2
    
    # Return the mean squared error
    return cost / len(circuits)

In [47]:
from scipy.optimize import minimize

# Initialize parameters
num_qubits = 8
# Generate random initial parameters for the variational quantum circuit
initial_params = np.random.rand(create_vqa_circuit(num_qubits).num_parameters)

# Train the model using the COBYLA optimization method to minimize the cost function
result = minimize(cost_function, initial_params, args=(estimator, X_train, y_train), method='COBYLA')
print("minimize:", result)

# Extract the trained parameters from the optimization result
trained_params = result.x

# Initialize the counter for correct predictions
correct_predictions = 0
counter = 0

# Test the model on the test set
for qc, label in zip(X_test, y_test):
    print("counter:", counter)
    counter += 1
    
    # Create the variational quantum circuit and assign the trained parameters
    vqc = create_vqa_circuit(qc.num_qubits)
    vqc.assign_parameters(trained_params, inplace=True)
    
    # Combine the test circuit with the variational circuit
    full_circuit = qc.compose(vqc)
    
    # Compute the expectation value of the Pauli-Z operator
    result = estimator.run([full_circuit], [SparsePauliOp("Z" * qc.num_qubits)]).result()
    
    # Determine the prediction based on the expectation value
    prediction = 0 if result.values[0] < 0 else 1
    
    # Check if the prediction matches the actual label
    if prediction == label:
        correct_predictions += 1

# Calculate the accuracy of the model on the test set
accuracy = correct_predictions / len(X_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

minimize:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.005129070281982422
       x: [ 1.524e+00 -5.252e-01 ...  6.775e-01  2.828e-01]
    nfev: 382
   maxcv: 0.0
counter: 0
counter: 1
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
Accuracy: 75.00%


# 4 qubit

In [48]:
import pennylane as qml
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# Number of qubits
num_qubits = 4

# Set the overlap threshold
mu = 1  # Desired maximum overlap

# Define the quantum devices
dev1 = qml.device('default.qubit', wires=num_qubits)
dev2 = qml.device('default.qubit', wires=num_qubits)

# Define the quantum circuit for Group 1 (Entangled States)
@qml.qnode(dev1)
def generate_entangled_state(params):
    qml.templates.AngleEmbedding(params, wires=range(num_qubits))
    # Apply CNOT gates to introduce entanglement
    for i in range(num_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.state()

# Define the quantum circuit for Group 2 (Product States)
@qml.qnode(dev2)
def generate_product_state(params):
    qml.templates.AngleEmbedding(params, wires=range(num_qubits))
    # No entangling gates applied here
    return qml.state()

# Function to generate states with overlap control
def generate_states_with_overlap_control(myu, num_states=10):
    group1 = []  # Entangled states
    group2 = []  # Product states

    while len(group1) < num_states or len(group2) < num_states:
        # Generate random parameters within distinct ranges
        params1 = np.random.uniform(0, np.pi/4, num_qubits)  # Narrow range for entangled states
        params2 = np.random.uniform(3 * np.pi / 4, np.pi, num_qubits)  # Different range for product states

        # Generate states
        psi1 = generate_entangled_state(params1)
        psi2 = generate_product_state(params2)

        # Compute overlap
        overlap = np.abs(np.vdot(psi1, psi2))**2

        # Check if the overlap condition is satisfied
        if overlap < myu:
            if len(group1) < num_states:
                group1.append(psi1)
            if len(group2) < num_states:
                group2.append(psi2)

    return group1, group2


# Convert PennyLane state vector to Qiskit QuantumCircuit
def state_to_quantum_circuit(state, num_qubits):
    # Convert PennyLane tensor to a NumPy array
    state_np = np.array(state)

    # Create a new QuantumCircuit with the same number of qubits
    qc = QuantumCircuit(num_qubits)
    
    # Initialize the QuantumCircuit with the given state vector
    qc.initialize(state_np, range(num_qubits))
    
    return qc

# Generate the states with the desired overlap
entangled_states, product_states = generate_states_with_overlap_control(myu=mu, num_states=200)

# Convert each entangled and product state into a QuantumCircuit
entangled_qcs = [state_to_quantum_circuit(psi, num_qubits) for psi in entangled_states]
product_qcs = [state_to_quantum_circuit(psi, num_qubits) for psi in product_states]

# Display the circuits
for i, qc in enumerate(entangled_qcs):
    print(f"Entangled Circuit {i + 1}:")
    print(qc)
    
for i, qc in enumerate(product_qcs):
    print(f"Product Circuit {i + 1}:")
    print(qc)


Entangled Circuit 1:
     »
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                   ├
«     │                                                                                                                                                                                    │
«q_1: ┤1                                                                                                                                                                                   ├
«     │  Initialize(0.8929,-0.3j,-0.10085,-0.30016j,-0.00034497,0.0001159j,-0.00034478,-0.0010262j,-0.00011526,3.8724e-05j,1.3018e-05,3.8745e-05j,-0.033713,0.01

In [49]:
entangled_qcs

In [50]:
X_train = entangled_qcs + product_qcs

y_train = np.array([1] * len(entangled_qcs) + [0] * len(product_qcs))


In [52]:
# Generate the states with the desired overlap
entangled_states_test, product_states_test = generate_states_with_overlap_control(myu=mu, num_states=10)

# Convert each entangled and product state into a QuantumCircuit
entangled_qcs_test = [state_to_quantum_circuit(psi, num_qubits) for psi in entangled_states_test]
product_qcs_test = [state_to_quantum_circuit(psi, num_qubits) for psi in product_states_test]

X_test = entangled_qcs_test + product_qcs_test

y_test = np.array([1] * len(entangled_qcs_test) + [0] * len(product_qcs_test))


In [57]:
print("X_train shape:", len(X_train))
print("X_test shape:", len(X_test))
print("Y_train shape:", len(y_train))
print("Y_test shape:", len(y_test))
print()

X_train shape: 400
X_test shape: 20
Y_train shape: 400
Y_test shape: 20



In [58]:
from qiskit.circuit.library import EfficientSU2
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp, Pauli


def create_vqa_circuit(num_qubits):
    return EfficientSU2(num_qubits, reps=1)


# Initialize Estimator
estimator = AerEstimator()

def cost_function(params, estimator, circuits, labels):
    """
    Define the cost function for the VQA.

    Args:
        params (array): Parameters for the variational quantum circuit.
        estimator (Estimator): Quantum estimator to compute expectation values.
        circuits (list): List of quantum circuits representing the training data.
        labels (list): List of actual labels for the training data.

    Returns:
        float: The mean squared error between the expectation values and the labels.
    """
    cost = 0
    for qc, label in zip(circuits, labels):
        # Create the variational quantum circuit and assign parameters
        vqc = create_vqa_circuit(qc.num_qubits)
        vqc.assign_parameters(params, inplace=True)
        
        # Combine the training circuit with the variational circuit
        full_circuit = qc.compose(vqc)
        
        # Compute the expectation value of the Pauli-Z operator
        result = estimator.run([full_circuit], [SparsePauliOp("Z" * qc.num_qubits)]).result()
        expectation_value = result.values[0]
        
        # Calculate the squared difference between the expectation value and the label
        cost += (expectation_value - label) ** 2
    
    # Return the mean squared error
    return cost / len(circuits)

In [59]:
from scipy.optimize import minimize

# Initialize parameters
# Generate random initial parameters for the variational quantum circuit
initial_params = np.random.rand(create_vqa_circuit(num_qubits).num_parameters)

# Train the model using the COBYLA optimization method to minimize the cost function
result = minimize(cost_function, initial_params, args=(estimator, X_train, y_train), method='COBYLA')
print("minimize:", result)

# Extract the trained parameters from the optimization result
trained_params = result.x

# Initialize the counter for correct predictions
correct_predictions = 0
counter = 0

# Test the model on the test set
for qc, label in zip(X_test, y_test):
    print("counter:", counter)
    counter += 1
    
    # Create the variational quantum circuit and assign the trained parameters
    vqc = create_vqa_circuit(qc.num_qubits)
    vqc.assign_parameters(trained_params, inplace=True)
    
    # Combine the test circuit with the variational circuit
    full_circuit = qc.compose(vqc)
    
    # Compute the expectation value of the Pauli-Z operator
    result = estimator.run([full_circuit], [SparsePauliOp("Z" * qc.num_qubits)]).result()
    
    # Determine the prediction based on the expectation value
    prediction = 0 if result.values[0] < 0 else 1
    
    # Check if the prediction matches the actual label
    if prediction == label:
        correct_predictions += 1

# Calculate the accuracy of the model on the test set
accuracy = correct_predictions / len(X_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

minimize:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.03005443572998047
       x: [ 1.383e+00  1.742e+00 ...  1.488e+00  1.505e-01]
    nfev: 186
   maxcv: 0.0
counter: 0
counter: 1
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
Accuracy: 85.00%
